<a href="https://colab.research.google.com/github/rahiakela/mlops-research-and-practice/blob/main/machine-learning-engineering-with-python/03-model-factory/02_data_drift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

In [ ]:
!pip install alibi
!pip install alibi-detect

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
from sklearn import metrics
from sklearn.pipeline import make_pipeline

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import alibi
from alibi_detect.cd import ChiSquareDrift, TabularDrift
from alibi_detect.utils.saving import save_detector, load_detector

##Dataset

In [3]:
# Grab the data
wine_data = load_wine()
feature_names = wine_data.feature_names
X, y = wine_data.data, wine_data.target

# Make a 50/50 reference/test split
X_ref, X_test, y_ref, y_test = train_test_split(X, y,
                                                test_size=0.50,
                                                random_state=42)

##Drift Detector

In [6]:
TabularDrift?

In [7]:
# Initialise the detector
cd = TabularDrift(p_val=0.05, x_ref=X_ref)

/usr/local/lib/python3.10/dist-packages/alibi_detect/cd/tabular.py:113: UserWarning: No `categories_per_feature` dict provided so all features are assumed to be numerical. `KSDrift` will be applied to all features.
  warnings.warn('No `categories_per_feature` dict provided so all features are assumed to be numerical. '


In [8]:
# Check for drift
pred = cd.predict(X_test)

labels = ["No", "Yes"]
print(f"Drift: {labels[pred['data']['is_drift']]}")

Drift: No


In [9]:
# Check for drift - X_test_cal_error is simulated calibration error test set
X_test_cal_error = 1.1 * X_test
pred = cd.predict(X_test_cal_error)

labels = ["No", "Yes"]
print(f"Drift: {labels[pred['data']['is_drift']]}")

Drift: Yes


##Feature drift

In [10]:
# for drift in the features, we will use the initial label as our baseline dataset
cd = TabularDrift(x_ref=y_ref, p_val=.05)

/usr/local/lib/python3.10/dist-packages/alibi_detect/cd/tabular.py:113: UserWarning: No `categories_per_feature` dict provided so all features are assumed to be numerical. `KSDrift` will be applied to all features.
  warnings.warn('No `categories_per_feature` dict provided so all features are assumed to be numerical. '


In [11]:
# check for drift in the test labels against the reference dataset
pred = cd.predict(y_test)

labels = ["No", "Yes"]
print(f"Drift: {labels[pred['data']['is_drift']]}")

Drift: No


In [12]:
# we can simulate some drift in the data, and then check that this is indeed detected
y_test_cal_error = 1.1 * y_test
pred = cd.predict(y_test_cal_error)

labels = ["No", "Yes"]
print(f"Drift: {labels[pred['data']['is_drift']]}")

Drift: Yes
